## Preprocessing

In [90]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import random
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [91]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=['NAME','EIN']) 

In [92]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [93]:
# Look at APPLICATION_TYPE value counts for binning
counts=application_df['APPLICATION_TYPE'].value_counts()
counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [94]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=list(counts[counts < 600].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [95]:
# Look at CLASSIFICATION value counts for binning
class_counts=application_df.CLASSIFICATION.value_counts()

In [96]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.CLASSIFICATION.value_counts() > 1

CLASSIFICATION
C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C4120    False
C8210    False
C2561    False
C4500    False
C2150    False
Name: count, Length: 71, dtype: bool

In [97]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 500].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [98]:
# Convert categorical data to numeric with `pd.get_dummies`
application_with_dummies_df = pd.get_dummies(application_df).astype(int) # there is a quirk of my enviorment which has get dummies outputing boolean values instead of 0 or 1.


In [99]:
# Split our preprocessed data into our features and target arrays
X=application_with_dummies_df.drop(['IS_SUCCESSFUL'], axis=1)
y=application_with_dummies_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [100]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [101]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_features = len(X_train_scaled[0])
hidden_nodes1=100
hidden_nodes2=50
hidden_nodes3=25

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='tanh'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = optimizers[optimizers_select], metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 100)               4400      
                                                                 
 dense_118 (Dense)           (None, 50)                5050      
                                                                 
 dense_119 (Dense)           (None, 25)                1275      
                                                                 
 dense_120 (Dense)           (None, 1)                 26        
                                                                 
Total params: 10751 (42.00 KB)
Trainable params: 10751 (42.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5661 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7319
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7329
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5457 - accuracy: 0.7337
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7343
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5445 - accura

In [102]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_features = len(X_train_scaled[0])
hidden_nodes1=100
hidden_nodes2=50
hidden_nodes3=25

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = optimizers[optimizers_select], metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_121 (Dense)           (None, 100)               4400      
                                                                 
 dense_122 (Dense)           (None, 50)                5050      
                                                                 
 dense_123 (Dense)           (None, 25)                1275      
                                                                 
 dense_124 (Dense)           (None, 1)                 26        
                                                                 
Total params: 10751 (42.00 KB)
Trainable params: 10751 (42.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5671 - accuracy: 0.7195
Epoch 2/100
804/804 [===========================

In [103]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_features = len(X_train_scaled[0])
hidden_nodes1=100
hidden_nodes2=50
hidden_nodes3=25

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = optimizers[optimizers_select], metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_125 (Dense)           (None, 100)               4400      
                                                                 
 dense_126 (Dense)           (None, 50)                5050      
                                                                 
 dense_127 (Dense)           (None, 25)                1275      
                                                                 
 dense_128 (Dense)           (None, 1)                 26        
                                                                 
Total params: 10751 (42.00 KB)
Trainable params: 10751 (42.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5681 - accuracy: 0.7220
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5501 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7342
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7344
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5450 - accuracy: 0.7350
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5445 - accura

In [104]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_features = len(X_train_scaled[0])
hidden_nodes1=150
hidden_nodes2=100
hidden_nodes3=30

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = optimizers[optimizers_select], metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=125)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_129 (Dense)           (None, 150)               6600      
                                                                 
 dense_130 (Dense)           (None, 100)               15100     
                                                                 
 dense_131 (Dense)           (None, 30)                3030      
                                                                 
 dense_132 (Dense)           (None, 1)                 31        
                                                                 
Total params: 24761 (96.72 KB)
Trainable params: 24761 (96.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/125
804/804 [==============================] - 2s 1ms/step - loss: 0.5679 - accuracy: 0.7207
Epoch 2/125
804/804 [===========================

In [106]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

input_features = len(X_train_scaled[0])
hidden_nodes1=85
hidden_nodes2=55
hidden_nodes3=30

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = optimizers[optimizers_select], metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=125)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_133 (Dense)           (None, 85)                3740      
                                                                 
 dense_134 (Dense)           (None, 55)                4730      
                                                                 
 dense_135 (Dense)           (None, 30)                1680      
                                                                 
 dense_136 (Dense)           (None, 1)                 31        
                                                                 
Total params: 10181 (39.77 KB)
Trainable params: 10181 (39.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/125
804/804 [==============================] - 2s 1ms/step - loss: 0.5680 - accuracy: 0.7201
Epoch 2/125
804/804 [===========================

In [107]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_opt.h5')